In [1]:
#!/usr/bin/env pytho
import unitymouse.render as umr
import time
import numpy as np
from ibllib.atlas.regions import BrainRegions
from one.api import ONE
import ibllib.atlas as atlas
import pickle

"""
Display all trajectories in one single brain volume
Author: Mayo, Gaelle, Dan
"""

import numpy as np

one = ONE()
ba = atlas.AllenAtlas(25)
traj_rep = one.alyx.rest('trajectories', 'list', provenance='Planned',
    x=-2243, y=-2000,  project='ibl_neuropixel_brainwide_01', use_cache=False)


Downloading: C:\Users\Dan\Downloads\ONE\alyx.internationalbrainlab.org\tmp7oqukby1\cache.zip Bytes: 100435669


1258063573it [00:14, 86932148.30it/s]                                                                                  


In [6]:
data = {}

for traj in traj_rep:

    if traj['probe_insertion'] not in except_pid:
        temp_traj = one.alyx.rest('trajectories', 'list',
            provenance='Ephys aligned histology track',
            probe_insertion=traj['probe_insertion'], use_cache=False)
        if len(temp_traj) == 0:
            temp_traj = one.alyx.rest('trajectories', 'list', provenance='Histology track',
                probe_insertion=traj['probe_insertion'], use_cache=False)

            if len(temp_traj) == 0:
                continue

        if not temp_traj[0]['x']:
            continue

        ins = atlas.Insertion.from_dict(temp_traj[0])

        entry_coords = ba.xyz2ccf(ins.entry)
        tip_coords = ba.xyz2ccf(ins.tip)
        recording_size = np.sqrt(np.sum(np.power(entry_coords-tip_coords,2)))
        tip_angles = [ins.phi, ins.theta, ins.beta]
        data[traj['probe_insertion']] = (tip_coords, tip_angles, recording_size)
    
pickle.dump(data,open( './ibl_repro_ephys_traj.pkl', "wb" ))

In [2]:
umr.setup()

UnityMouse Renderer connected to server
Login sent with ID: Dan


In [3]:
data = pickle.load( open( "./ibl_repro_ephys_traj.pkl", "rb" ) )


In [8]:
umr.clear()

In [9]:
# TODO removing PID manually for sake of figure, but need to iron this out in Alyx
except_pid = ['8b735d77-b77b-4243-8821-37802bf402fe',
'94af9073-0914-4323-a90a-5eea1ef5f92c']

# temp_traj_rep = [traj_rep[0]]
count = 0

for i, (pid, traj) in enumerate(data.items()):
    tip_coords = traj[0]
    tip_angles = traj[1]
    recording_size = traj[2]
    
    probename = 'p'+str(count)
    count+=1
    umr.create_probes([probename])
    umr.set_probe_positions({probename:tip_coords.tolist()})
    umr.set_probe_colors({probename:'#F28C28'})
    umr.set_probe_angles({probename:tip_angles})
    umr.set_probe_size({probename:[0.07,recording_size/1000,0.02]})


'''
Display structure meshes within the brain volume
You can download the mesh object for each brain structure here:
http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/structure_meshes/
'''
br = BrainRegions()

vdata = {'VISa': True, 'CA1': True, 'DG': True, 'LP': True, 'PO': True}
umr.set_volume_visibility(vdata)
vdata = {'VISa': 'left', 'CA1': 'left', 'DG': 'left', 'LP': 'left', 'PO': 'left'}
umr.set_volume_style(vdata)
vdata = {'VISa': 'transparent', 'CA1': 'transparent', 'DG': 'transparent', 'LP': 'transparent', 'PO': 'transparent'}
umr.set_volume_shader(vdata)
vdata = {'VISa': 0.4, 'CA1': 0.4, 'DG': 0.4, 'LP': 0.4, 'PO': 0.4}
umr.set_volume_alpha(vdata)

# data = {'root':True}
# umr.set_volume_visibility(data);



# target_area = ['VISa', 'CA1', 'DG', 'LP', 'PO']
# for target in target_area:
#     rindx_void = np.where(br.acronym == target)
#     idx = rindx_void[0][0]
#     mesh_id = br.id[idx]
#     # print(mesh_id) --> useful to download the specific mesh obj from the Allen website
#     color = br.rgb[idx, :]/255
#     path = f'/Users/gaelle/Desktop/Allenmesh/{mesh_id}.obj.txt'
#     rendering.add_mesh(fig, path, color, opacity=0.6)



In [15]:
umr.close()

UnityMouse Renderer connected to server
Login sent with ID: Dan
